# Training a text generator with WordVec-RNN 

In this notebook we will look at how to train a Recurrent Neural Network (RNN) to model sequence of words vectors (WordVec-RNN). This is somewhat similiar to the notebooks modelling sequences of characters and words, but there are some important differences. The size to the input of the model is determined by the embedding dimension of the word vectors (which is a fixed size), not the number of words or characters in the dataset. This means that the RNN can be fed in words that are not in the training dataset, and can also be configured to predict words it has not seen (though this current implementation [does limit the generation only to words in the training set](#masking-for-data-only-in-the-dataset), we can turn that off later).

To do this we will use the library [PyTorch](https://pytorch.org/), a library for building and training neural networks in Python. This is the first time we are using this library and looking at how to build and train neural networks, but it won't be the last! We will be spending a lot of time over the next couple of terms looking at code that looks a lot like this. It may look quite unfamiliar at first, but over time you will get used to working with and altering this kind of code. 

First lets do some imports:

In [ ]:
import os
import torch
import random

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchtext.vocab as vocab

from torchtext.data.utils import get_tokenizer
from torch.distributions import Categorical

In PyTorch, there are different implementations for storing and processing data on different kinds of computer hardware. By default, all computers will work by training and running neural networks on the Central Processing Unit (CPU), which we can specify with `'cpu'`. 

If you have an NVIDIA Graphics Processing Unit (GPU) (and you have installed CUDA correctly and the correct version of PyTorch), then you can use the flag `'gpu'` which will make training your neural networks **much faster**. Most of you won't have powerful NVIDIA GPU's in yor laptops however. Don't worry if you don't, the notebooks we are using in this class will be designed to work on laptop CPU's. 

If you have an M1 or M2 processor on a Mac then you can use the device `'mps'` which will run on Apples accelerated Metal Performance Shaders (MPS) for potentially faster and more powerful training (though sometimes running on CPU can be faster). 

In [ ]:
device = 'cpu'

#### Set hyperparameters

This is where we specify our *hyperparameters*. This is where we set the parameters that determine the size of our neural network (`num_layers`,`hidden_size` and `batch_size`), how long we train the network for (`num_steps` and `step_len`) and how aggressively we train the network (the learning rate `lr`).  

`load_chk` is a boolean that determines whether we start training from the weights of an already trained model or start from scratch. If this is true, then the path to a model file should be specified in `load_path`. If you change the dataset (and have a different vocabulary size) or make changes to any other parts of the model between saving and loading a model then this will not work. 

In [ ]:
hidden_size = 512   # size of hidden state
batch_size = 100    # length of LSTM sequence
gen_seq_len = 50    # size of generated text sequence
step_len = 200      # number of training samples in each stem
num_layers = 3      # number of layers in LSTM layer stack
lr = 0.002          # learning rate
num_steps = 1000     # max number of training steps
load_chk = False    # load in pre-trained checkpoint for training
save_path = "wordvec_rnn_model.pt"
# load_path = "wordvec_rnn_model.pt"

#### Load multiple text files

This function should now look familiar. We will use it to load in our Nursery Rhymes dataset. As we are loading in all of the files into one string variable we need a way to determine where one rhyme ends and another begins. We will do this by adding in the word `EOF` (End of File) which can will the model to represent where one rhyme ends and another begins. 

In [ ]:
def load_all_text_files_in_folder(path, max_files = 10000):
    corpus = ''
    # Find all files in the folder or subfolders
    for root, _, files in os.walk(path):
        for i, file in enumerate(files):
            # If the file is a text file
            if file.endswith(".txt") and i <= max_files:
                # Open the file and add the text to the corpus
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                    # Add text from file
                    corpus += text
                    # Add 'End of File' between documents
                    corpus += '\n EOF \n'
    return corpus

#### Load our data

Now lets load in our data. We are going to use a dataset of pokemon names to start off with. We are going to use the [Python set data structure](https://www.w3schools.com/python/python_sets.asp) to find all of the unique words in our text. We will first use the `split()` function to split our corpus into words. 

In [ ]:
data_path = "../data/nursery-rhymes"
corpus = load_all_text_files_in_folder(data_path)
words = sorted(list(set(corpus.split())))
data_size, vocab_size = len(corpus.split()), len(words)

#### Load word vectors

Here we will load in our word vectors. By default we are going to download only the top30k words from GloVe, this is for speed and efficiency. You can come back to this later and try loading in some other word vectors.

A torchtext Vectors class has two dictionaries in it `stoi` (string to index) and `itos` (index to string). These are equivalent to the dictionaries we made in the previous notebook `word_to_ix` and `ix_to_word`. For consistency we are going to assign variables with the same name to make this and the Week 7 `word-rnn-training.ipynb` notebooks easier to compare. 

In [ ]:
# Uncomment the next line and use instead of the following line to use the full GloVe dictionary
# word_vectors = vocab.GloVe(name="6B",dim=100) 
word_vectors = vocab.Vectors(name = '../data/glove.6B.100d.top30k.txt')
tokenizer = get_tokenizer("basic_english")
wordvec_embeddings = nn.Embedding.from_pretrained(word_vectors.vectors)
embedding_dim = wordvec_embeddings.weight.shape[1]

# Get dictionaries from word_vectors class and 
# rename to be consistent with previous notebooks
word_to_ix = word_vectors.stoi
ix_to_word = word_vectors.itos

Now lets get our list of tokens form our text string using the torchtext `tokenizer` class. We will remove tokens that we do not have word vectors for. We will then create our `data` variable by getting the word vector for each token in our dataset in order.

In [ ]:
tokens = tokenizer(corpus) 
print(f'There raw data cosists of {len(tokens)} tokens.')
tokens = [token for token in tokens if token in word_to_ix]
print(f'There processed data cosists of {len(tokens)} tokens (that exist in the word vector vocab)')
data = word_vectors.get_vecs_by_tokens(tokens, lower_case_backup=True)

#### Masking for data only in the dataset

this next bit of code is going to be useful later. Essentially what we want to do is figure out all of our unique words, get the indexes for them in our `word_vectors` matrix. We will use this information to make a variable called `mask_array` which is a list the length of the number of words in our `word_vectors` matrix. At the corresponding index of where we have words in our dataset, the value will be `1`, for words not in our dataset, the value will be `0`. We can then use this to ensure that when we sample words we only sample words in our dataset. 

This is not essential but it will help to keep the generations from our model bounded to the vocabularly of the dataset. 

In [ ]:
data_vocab = sorted(list(set(tokens)))
print(data_vocab)
data_indexes = [word_to_ix[word] for word in data_vocab]
print(data_indexes)
mask_array = np.zeros(len(word_to_ix), dtype=int)
mask_array[data_indexes] = 1
print(mask_array)
mask_array = torch.tensor(mask_array, dtype=torch.int64, device=device)

#### Defining the network 

This is where we define our neural network. We define a neural network as a `class`. Classes can have **functions** and ****variables** that it owns. Classes are a fundamental part of [object-orientated programming in Python](https://www.w3schools.com/python/python_classes.asp) (and many other programming languages). In this case we are building our class `RNN` by [inheriting from the base class](https://www.w3schools.com/python/python_inheritance.asp) for a neural network in PyTorch called `nn.Module`. 

We need to define two functions for a PyTorch neural network class. The `__init__` function gets called when we create the class, here we create and set the variables that our network needs (such as the all of layers and other things we may need to keep track of). In this function the first thing we need to call is the `super` function that will call the `__init__` function of the base class we are inheriting from. 

The other function we need to define for a PyTorch neural networks is the function `forward` function. This defines what happens when we do a forward pass with our network (taking data as an input and giving somethign else as an output). Because this is a recurrent neural network, our network needs to take as input both the data and the hidden state (the previous iteration) of the model. This function also outputs the hidden state so that we can pass it back into the function at a later iteration:


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()
    
    def forward(self, input_batch, hidden_state):
        output, hidden_state = self.rnn(input_batch, hidden_state)
        output = self.decoder(output)
        output = self.tanh(output)
        return output, (hidden_state[0].detach(), hidden_state[1].detach())

#### Setting up network and optimiser

Here we will create an instantiation of our network `rnn`. We also need to define out loss function `loss_fn` and our `optimiser`, which is used to make make changes to the neural network weights in training. We have to make our data variable a PyTorch `tensor`. This is data type that we have to use with PyTorch so that our neural networks can read and process the data correctly. [PyTorch tensors](https://pytorch.org/docs/stable/tensors.html) have been designed to work in almost exactly the same way as [numpy arrays](https://numpy.org/doc/stable/reference/generated/numpy.array.html).

In [ ]:
# Create list of indexes that can be valid starting points for training
index_list = list(range(0, len(data) - step_len - 1))

# Conver data to torch tensor
data = torch.tensor(data).to(device)
data = torch.unsqueeze(data, dim=1)

# Create RNN class
rnn = RNN(embedding_dim, embedding_dim, hidden_size, num_layers).to(device)

# Define loss function and optimiser
loss_fn = nn.CosineSimilarity()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

if load_chk:
    checkpoint = torch.load(load_path)
    rnn.load_state_dict(checkpoint['state_dict'])

#### Sample data randomly
This function will allow us to do random sampling of the dataset in training. When we train neural networks we almost always train on **random batches of data**. In training we process lots of data samples at the same time (with lots of copies of the neural network), and then average our loss over all of the data samples and update the weights accordingly. This helps with the *regularisation* of the network, and makes training much more stable. 

The number of data samples we have in each mini-batch is defined by the `batch_size`, generally speaking the more batches you can use the better (though there are exceptions to this).

In [ ]:
def get_training_batch_indicies(index_list, batch_size):
    # Get a batch of indicies to sample our data from
    input_batch_indicies = torch.tensor(np.array(random.sample(index_list, batch_size)))
    # Offset indicies for target batch by one
    target_batch_indicies = input_batch_indicies + 1
    return input_batch_indicies, target_batch_indicies

#### Training the network

Here we have all the code we need for our **training loop**. Here we are looping through our number of training steps (defined in `num_steps`). Each step we will sample a random section of a dataset that will cycle for `step_len` training iterations. In some code bases, you will see code that cycles through *epochs* (complete cycles of the dataset). We aren't doing that here as the training time can vary drastically based on how much data is in your dataset. 

After each iteration the weights of the model will be saved to the file `wordvec_rnn_model.pt`. Sometimes people will save different versions of the file after a set number of step (i.e. `wordvec_rnn_model_50.pt` or `wordvec_rnn_model_100.pt`), but this will fill up your drive very quickly! For now we will just keep overwriting the same file after each iteration. 

If you are happy with the outputs or need to stop the code running for whatever reason, you can just kill the cell and your progress will be saved. This can be loaded into the notebook `wordvec-rnn-text-gen-test.ipynb` to be used for code that just performs generation. 

The most important parts of any training code are the **forward pass** (where we process our data with our neural network), calculating the **loss function** where evaluate how well our model has performed against the real value of the data. Then we have to **update the weights of the neural network**. This is done by calling `loss.backward()` followed by `optimizer.step()`.

After each iteration of the code we generate a new sequence with the network so we can see how the network is improving during training. This is done without gradient tracking `with torch.no_grad():` (gradient tracking is what is used for training and calculating how much to adjust the weights of our network by at each step). 

This will probably all look quite complicated and hard to understand first time around. **That is ok!** Over time as you see and work with more and more code that looks like this you will start getting used to it and feel more confident in adapting, changing and writing this kind of code yourself.


In [ ]:
# Iterate through the number of steps defined earlier
for step in range(1, num_steps):
    
    running_loss = 0
    hidden_state = None
    rnn.zero_grad()
    train_batch_indicies, target_batch_indicies = get_training_batch_indicies(index_list, batch_size)
    # Cycle through for a set number of consecutive iterations in the data
    for i in range(step_len):
        input_batch = data[train_batch_indicies].squeeze()
        target_batch = data[target_batch_indicies].squeeze()
    
        # Forward pass
        # The following code is the same as calling rnn.forward(input_batch, hidden_state)
        output, hidden_state = rnn(input_batch, hidden_state)
        
        # Compute loss (we take 1 minus the loss as it is measure similiarity not distance)
        loss = 1 - loss_fn(output, target_batch).mean() 
        running_loss += loss.item() / step_len
        # print(f'loss {loss}, running loss {running_loss}')
        
        # Update weights of neural network
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Increment batch indicies by 1
        train_batch_indicies = train_batch_indicies + 1
        target_batch_indicies = target_batch_indicies + 1
        

        
    # Print loss
    print('\n'+'-'*75)
    print(f"\nStep: {step} Loss: {running_loss}")

    # Create a dictionary for saving the model and data mappings
    save_dict = {}
    # Add the model weight parameters as a dictionary to our save_dict
    save_dict['state_dict'] = rnn.state_dict()
    # Add the idx_to_char and char_to_idx dicts to our save_dict
    save_dict['ix_to_word'] = ix_to_word
    save_dict['word_to_ix'] = word_to_ix
    # Save the dictionary to a file
    torch.save(save_dict, save_path)

    # Now lets generate a random generated text sample to print out,
    # we will do this without gradient tracking as we are not training
    with torch.no_grad():
        
        # Take a random index and reset the hidden state of the model
        rand_index = np.random.randint(data_size-1)
        input = data[rand_index : rand_index+1].squeeze().unsqueeze(0)
        hidden_state = None
        
        # Iterate over our sequence length
        for i in range(gen_seq_len):
            
            # Forward pass
            output, hidden_state = rnn(input, hidden_state)

            # Comput distances to all words
            dists = torch.norm(word_vectors.vectors - output[0], dim=1) 
            # Use softmax to convert to probabilities
            probs = F.softmax(1 - dists, dim=0)
            # Multiply probabilities by mask to only sample words from dataset
            probs = probs * mask_array
            # Covert probabilities to probability distribution
            prob_dist = Categorical(probs)
            # Sample from probability distribution
            word_index  = prob_dist.sample()

            # Get the next word and print
            next_word = ix_to_word[word_index]
            print(next_word, end=' ')
            
            # The word vector for the next word is the next input
            input = word_vectors[next_word].unsqueeze(0)

## Tasks 

First do these tasks in order before moving onto the bonus tasks:

**Task 1:** Run the code cells and train a model on the Nursery Rhymes names. Is it possble after training to generate plausible looking nursery rhymes. If the model training has finished and you are not happy with the results you can set `load_chk` to `True` in [the cell that defines the hyperparameters](#set-hyperparameters) and load in a model from the variable `load_path`.

**Task 2:** Load in a trained model into the `word-vector-rnn-text-gen-testing.ipynb` notebook. Play around with the `temperature` parameter and turn off the dataset mask to see how that effects the results.

**Task 3:** Compare the code in this notebook to the Word-RNN training (`Week7-Generating-text-with-neural-networks/word-rnn-training.ipynb`). How does the code differ? What elements are the same between the notebooks.

**Task 4:** Adapt this code to load in another dataset. Have a look at the code in `text-generation-with-markov-chains.ipynb` from Week 5 to use functions to load in datasets in different formats. Are the results better on datasets that uses more standard Englsh than the nursery rhymes dataset?

### Bonus tasks

These bonus tasks can be done in any order.

After each training run you may want to rename the checkpoint files from each training run so you can keep them for comparison later.

**Task A:** Try [changing the word vectors used in this code](#load-word-vectors). Does using a larger set of word vectors, a different dimensionality, or [a completely different set of word vectors](https://torchtext.readthedocs.io/en/latest/vocab.html#pretrained-word-embeddings) make any significiant qualititaive changes to the output or behaviour in training?

**Task B:** Try changing some of the other hyperparameters in [the cell that defines the hyperparameters](#set-hyperparameters). Such as `hidden_size` `seq_len`, `num_layers` or `lr`. Restart the kernel and run the training again. 

**Task C:** Try changing the optimiser used [the cell where the network and optimiser are instantiated](#setting-up-network-and-optimiser) to one of [the other optimisers available in PyTorch](https://pytorch.org/docs/stable/optim.html), such as stochastic gradient descent (SGD) or Adagrad. Restart the kernel and run the training again. 

**Task D:** Try changing the type of layer used [in the RNN network](#defining-the-network) from LSTM to a [vanilla RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN) or a [Gated Recurrent Unit](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU) (GRU). Restart the kernel and run the training again. 

**Task E:** Can you use stop words or do some other kinds of cleaning or normalisation to the dataset to improve or edit the quality of the generated results? 
